<a href="https://colab.research.google.com/github/Alimustoofaa/Train-NER-Container-Number-Doccano-Spacy/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy

In [1]:
import json

# spacy library
import spacy
import random
from spacy.util import minibatch, compounding
from spacy import load, displacy

In [2]:
def conversion(path):
  doc = open(path, "r")
  train_data = []
  try:
    lines = doc.readlines()
    for texts in lines:
      texts = json.loads(texts)
      text_content = texts['text']
      text_entities = texts['labels']
      entity_arr = []
      for entity in text_entities:
        entity = tuple(entity)
        entity_arr.append(entity)
      data = (text_content, {'entities': entity_arr})
      train_data.append(data)
  finally:
    doc.close()
  return train_data

In [3]:
# Conversion json1
TRAIN_DATA = conversion('datasets.json1')

In [4]:
# Get label
label_arr = []
for data in TRAIN_DATA:
  if len(data) == 2:
    entities = data[1]['entities']
    for entity in entities:
      label_arr.append(entity[2])

In [5]:
nlp=spacy.blank("id")
nlp.add_pipe(nlp.create_pipe('ner'))
nlp.begin_training()

In [ ]:
ner=nlp.get_pipe("ner")
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [10]:
for label in label_arr:
  ner.add_label(label)

In [11]:
with nlp.disable_pipes(*unaffected_pipes):
  for iteration in range(100):
    random.shuffle(TRAIN_DATA)
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    loss = {}
    for batch in batches:
      text, anotations = zip(*batch)
      nlp.update( text, anotations, drop=0.5, losses=loss)
    print(f'Losses at iteration {iteration}, {loss}')

Losses at iteration 0, {'ner': 510.7381180524826}
Losses at iteration 1, {'ner': 297.28984249476343}
Losses at iteration 2, {'ner': 308.16749531030655}
Losses at iteration 3, {'ner': 277.7352916970849}
Losses at iteration 4, {'ner': 275.99991785921156}
Losses at iteration 5, {'ner': 283.9891903405078}
Losses at iteration 6, {'ner': 268.1364307250815}
Losses at iteration 7, {'ner': 262.80059533362646}
Losses at iteration 8, {'ner': 267.19793466823467}
Losses at iteration 9, {'ner': 250.72067038999103}
Losses at iteration 10, {'ner': 258.7851408908573}
Losses at iteration 11, {'ner': 247.61882581259124}
Losses at iteration 12, {'ner': 244.54806064908604}
Losses at iteration 13, {'ner': 245.03285707471832}
Losses at iteration 14, {'ner': 247.02332231648325}
Losses at iteration 15, {'ner': 240.13894300106276}
Losses at iteration 16, {'ner': 234.6988903524059}
Losses at iteration 17, {'ner': 228.6629315328273}
Losses at iteration 18, {'ner': 223.59973881796296}
Losses at iteration 19, {'ner

In [12]:
# test 
doc = nlp("GATE IN  TCLU  jancuk TAL 45G1 3N15 928181 MA755 1L536 H L9US 508 B4IG TI  DYG III L3")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
print("Container Number :", ''.join([(i.text) for i in doc.ents]))

Entities [('TCLU', 'owner'), ('928181', 'digit')]
Container Number : TCLU928181


In [13]:
spacy.displacy.render(doc, style="ent", jupyter=True, options={'distance': 90})

In [14]:
# save model
from pathlib import Path
from datetime import datetime

datetime_now = datetime.now()
str_datetime = datetime_now.strftime("%m-%d-%Y %H:%M:%S")
output_dir = Path(f'nlp_container_number_checkpoint_{str_datetime}')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to nlp_container_number_checkpoint_12-17-2020 14:27:50


In [15]:
# load existing model 
# output_dir = 'nlp_id_checkpoint_2020_12_15'
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)

Loading from nlp_container_number_checkpoint_12-17-2020 14:27:50


In [16]:
doc = nlp_updated("GATE IN  TCLU TAL 45G1 3N15 928181 MA755 1L536 H L9US 508 B4IG TI  DYG III L3")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
print("Container Number :", ''.join([(i.text) for i in doc.ents]))

Entities [('TCLU', 'owner'), ('928181', 'digit')]
Container Number : TCLU928181
